<a href="https://colab.research.google.com/github/andresum97/03MAIR_algoritmos_de_optimizacion/blob/main/Actividad_Guiada_2_AG2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AG2 - Actividad Guiada 2
Nombre: Andrés Urízar

Link: https://colab.research.google.com/drive/1Ctn3beIWHK1anmbFmlxBE-JpGuLt5T4B?usp=sharing

Github: https://github.com/andresum97/03MAIR_algoritmos_de_optimizacion

### Programación dinámica

In [1]:
import math

In [2]:
#Viaje por el rio

TARIFAS = [
[0,5,4,3,float("inf"),float("inf"),float("inf")],   #desde nodo 0
[float("inf"),0,float("inf"),2,3,float("inf"),11],  #desde nodo 1
[float("inf"),float("inf"), 0,1,float("inf"),4,10], #desde nodo 2
[float("inf"),float("inf"),float("inf"), 0,5,6,9],
[float("inf"),float("inf"), float("inf"),float("inf"),0,float("inf"),4],
[float("inf"),float("inf"), float("inf"),float("inf"),float("inf"),0,3],
[float("inf"),float("inf"),float("inf"),float("inf"),float("inf"),float("inf"),0]
]

def Precios(TARIFAS):
################################################################ 
  #Total de Nodos
  N = len(TARIFAS[0])
  
  #Inicialización de la tabla de precios
  PRECIOS = [ [9999]*N for i in [9999]*N]  #n x n 
  RUTA = [ [""]*N for i in [""]*N]
  
  #Se recorren todos los nodos con dos bucles(origen - destino)
  #  para ir construyendo la matriz de PRECIOS
  for i in range(N-1):
    for j in range(i+1, N):
      MIN = TARIFAS[i][j]
      RUTA[i][j] = i
      
      for k in range(i, j):
        if PRECIOS[i][k] + TARIFAS[k][j] < MIN:
            MIN = min(MIN, PRECIOS[i][k] + TARIFAS[k][j] )
            RUTA[i][j] = k
        PRECIOS[i][j] = MIN
        
  return PRECIOS,RUTA

PRECIOS, RUTA = Precios(TARIFAS)
print(PRECIOS)
print(RUTA)

[[9999, 5, 4, 3, 8, 8, 11], [9999, 9999, inf, 2, 3, 8, 7], [9999, 9999, 9999, 1, 6, 4, 7], [9999, 9999, 9999, 9999, 5, 6, 9], [9999, 9999, 9999, 9999, 9999, inf, 4], [9999, 9999, 9999, 9999, 9999, 9999, 3], [9999, 9999, 9999, 9999, 9999, 9999, 9999]]
[['', 0, 0, 0, 1, 2, 5], ['', '', 1, 1, 1, 3, 4], ['', '', '', 2, 3, 2, 5], ['', '', '', '', 3, 3, 3], ['', '', '', '', '', 4, 4], ['', '', '', '', '', '', 5], ['', '', '', '', '', '', '']]


In [ ]:
#Calculo de la ruta usando la matriz RUTA
def calcular_ruta(RUTA, desde, hasta):
  if desde == RUTA[desde][hasta]:
    return desde 
  else:
    return str(calcular_ruta(RUTA, desde, RUTA[desde][hasta])) +  ',' + str(RUTA[desde][hasta]) 

print("\nLa ruta es:")  
calcular_ruta(RUTA, 0,6) 


La ruta es:


'0,2,5'

### Ramificación y Poda

In [2]:
#Asignacion de tareas - Ramificación y Poda
################################################################################
#    T A R E A 
#   A
#   G
#   E
#   N
#   T
#   E

COSTES=[[11,12,18,40],
        [14,15,13,22],
        [11,17,19,23],
        [17,14,20,28]]
        

In [3]:
#Calculo del valor de una solucion parcial
def valor(S,COSTES):
  VALOR = 0
  for i in range(len(S)):
    VALOR += COSTES[S[i]][i] 
  return VALOR



valor((3,2,  ),COSTES)

34

In [4]:
#Coste inferior para soluciones parciales
#  (1,3,) Se asigna la tarea 1 al agente 0 y la tarea 3 al agente 1

def CI(S,COSTES):
  VALOR = 0
  #Valores establecidos
  for i in range(len(S)):
    VALOR += COSTES[i][S[i]]

  #Estimacion
  for i in range( len(S), len(COSTES)   ):
    VALOR += min( [ COSTES[j][i] for j in range(len(S), len(COSTES))  ])
  return VALOR

def CS(S,COSTES):
  VALOR = 0
  #Valores establecidos
  for i in range(len(S)):
    VALOR += COSTES[i][S[i]]

  #Estimacion
  for i in range( len(S), len(COSTES)   ):
    VALOR += max( [ COSTES[j][i] for j in range(len(S), len(COSTES))  ])
  return VALOR

CI((0,1),COSTES)

68

In [5]:
#Genera tantos hijos como como posibilidades haya para la siguiente elemento de la tupla
#(0,) -> (0,1), (0,2), (0,3)
def crear_hijos(NODO, N):
  HIJOS = []
  for i in range(N ):
    if i not in NODO:
      HIJOS.append({'s':NODO +(i,)    })
  return HIJOS

In [6]:
crear_hijos((0,) , 4)

[{'s': (0, 1)}, {'s': (0, 2)}, {'s': (0, 3)}]

In [7]:
def ramificacion_y_poda(COSTES):
#Construccion iterativa de soluciones(arbol). En cada etapa asignamos un agente(ramas).
#Nodos del grafo  { s:(1,2),CI:3,CS:5  }
  DIMENSION = len(COSTES)  
  MEJOR_SOLUCION=tuple( i for i in range(len(COSTES)) )
  CotaSup = valor(MEJOR_SOLUCION,COSTES)


  NODOS=[]
  NODOS.append({'s':(), 'ci':CI((),COSTES)    } )

  iteracion = 0

  while( len(NODOS) > 0):
    iteracion +=1

    nodo_prometedor = [ min(NODOS, key=lambda x:x['ci']) ][0]['s']

    #Ramificacion
    #Se generan los hijos
    HIJOS =[ {'s':x['s'], 'ci':CI(x['s'], COSTES)   } for x in crear_hijos(nodo_prometedor, DIMENSION) ]

    #Revisamos la cota superior y nos quedamos con la mejor solucion si llegamos a una solucion final
    NODO_FINAL = [x for x in HIJOS if len(x['s']) == DIMENSION  ]
    if len(NODO_FINAL ) >0: 
      if NODO_FINAL[0]['ci'] < CotaSup:
        CotaSup = NODO_FINAL[0]['ci']
        MEJOR_SOLUCION = NODO_FINAL
 
    #Poda
    HIJOS = [x for x in HIJOS if x['ci'] < CotaSup   ]

    #Añadimos los hijos 
    NODOS.extend(HIJOS) 

    #Eliminamos el nodo ramificado
    NODOS =  [  x for x in NODOS if x['s'] != nodo_prometedor    ]
   
  print("La solucion final es:" ,MEJOR_SOLUCION , " en " , iteracion , " iteraciones" , " para dimension: " ,DIMENSION  )
      


ramificacion_y_poda(COSTES)

La solucion final es: [{'s': (1, 2, 0, 3), 'ci': 64}]  en  10  iteraciones  para dimension:  4


### Algoritmo Fuerza Bruta

In [8]:
#Algoritmo de Fuerza Bruta
from itertools import permutations

def fuerza_bruta(costes):
  n = len(costes)
  cost = 9999999
  s = []
  combinations = list(permutations([i for i in range(n)]))
  for element in combinations:
    cost_temp = 0
    for idx, value in enumerate(element):
      cost_temp += costes[idx][value]

    if cost_temp < cost:
      cost = cost_temp
      s = element

  return [{'s': s, 'ci': cost}]


res = fuerza_bruta(COSTES)
res

[{'s': (0, 2, 3, 1), 'ci': 61}]

In [9]:
# Valores aleatorios
from random import randint
from timeit import default_timer as timer
from tabulate import tabulate

COSTES_5 = [[randint(10,40) for _ in range(5)] for _ in range(5)]
COSTES_6 = [[randint(10,40) for _ in range(6)] for _ in range(6)]
COSTES_7 = [[randint(10,40) for _ in range(7)] for _ in range(7)]
COSTES_8 = [[randint(10,40) for _ in range(8)] for _ in range(8)]
COSTES_10 = [[randint(10,40) for _ in range(10)] for _ in range(10)]
COSTES_12 = [[randint(10,40) for _ in range(12)] for _ in range(12)]
COSTES_13 = [[randint(10,40) for _ in range(13)] for _ in range(13)]

In [12]:
tiempos = [
    ['Costos', 'Tiempo Fuerza Bruta', 'Tiempo Ramificacion y Poda'],
    ['5'],
    ['6'],
    ['7'],
    ['8'],
    ['10'],
    ['12'],
    ['13']
]


# fUERZA BRUTA
# 5
start = timer()
fuerza_bruta(COSTES_5)
end = timer()
tiempos[1].append(end - start)

# 6
start = timer()
fuerza_bruta(COSTES_6)
end = timer()
tiempos[2].append(end - start)

# 7
start = timer()
fuerza_bruta(COSTES_7)
end = timer()
tiempos[3].append(end - start)

# 8
start = timer()
fuerza_bruta(COSTES_8)
end = timer()
tiempos[4].append(end - start)

# 10
start = timer()
fuerza_bruta(COSTES_10)
end = timer()
tiempos[5].append(end - start)

# 12
# start = timer()
# fuerza_bruta(COSTES_12)
# end = timer()
tiempos[6].append('na')

# 13
# start = timer()
# fuerza_bruta(COSTES_12)
# end = timer()
tiempos[7].append('na')


# RAMIFICACION Y PODA
# 5
start = timer()
ramificacion_y_poda(COSTES_5)
end = timer()
tiempos[1].append(end - start)

# 6
start = timer()
ramificacion_y_poda(COSTES_6)
end = timer()
tiempos[2].append(end - start)

# 7
start = timer()
ramificacion_y_poda(COSTES_7)
end = timer()
tiempos[3].append(end - start)

# 8
start = timer()
ramificacion_y_poda(COSTES_8)
end = timer()
tiempos[4].append(end - start)

# 10
start = timer()
ramificacion_y_poda(COSTES_10)
end = timer()
tiempos[5].append(end - start)

# 12
start = timer()
ramificacion_y_poda(COSTES_12)
end = timer()
tiempos[6].append(end - start)

# 13
start = timer()
ramificacion_y_poda(COSTES_13)
end = timer()
tiempos[7].append(end - start)


print(tabulate(tiempos, headers='firstrow', tablefmt='grid'))

La solucion final es: [{'s': (2, 4, 1, 3, 0), 'ci': 78}]  en  50  iteraciones  para dimension:  5
La solucion final es: [{'s': (0, 4, 2, 3, 5, 1), 'ci': 91}]  en  419  iteraciones  para dimension:  6
La solucion final es: [{'s': (0, 2, 3, 6, 1, 4, 5), 'ci': 104}]  en  90  iteraciones  para dimension:  7
La solucion final es: [{'s': (5, 4, 2, 7, 1, 0, 6, 3), 'ci': 131}]  en  792  iteraciones  para dimension:  8
La solucion final es: [{'s': (8, 2, 0, 3, 6, 1, 5, 4, 9, 7), 'ci': 118}]  en  25342  iteraciones  para dimension:  10
La solucion final es: [{'s': (2, 3, 4, 7, 0, 11, 10, 9, 5, 1, 6, 8), 'ci': 147}]  en  575  iteraciones  para dimension:  12
La solucion final es: [{'s': (2, 6, 0, 4, 5, 9, 1, 12, 8, 10, 7, 11, 3), 'ci': 164}]  en  33887  iteraciones  para dimension:  13
+----------+------------------------+------------------------------+
|   Costos | Tiempo Fuerza Bruta    |   Tiempo Ramificacion y Poda |
+==========+========================+==============================+
|      

Se hicieron multiples pruebas para esta parte, con varias matrices de diferentes dimensiones, el tiempo solo fue para ver si habia una diferencia significativa entre algoritmos. El problema fue que con valores muy altos de dimensión Google Colab se trababa por su RAM. Sin embargo se pudieron hacer distintas pruebas donde se comprobó que luego de la dimensión 10 fuerza bruta ya se era muy complicado de manejar, entonces se asume que ese es su limite, mientras que con ramificación y poda, este fue tuvo un mayor rango de la dimensión, que fue 12. Por lo que se puede ver en la tabla, depende mucho de la cantidad de iteraciones, como se ve con 10 que realizó una enorme cantidad de iteraciones, pero luego de 12 el valor subió demasiado, pasando los 2 minutos de tiempo, indicando que ya para esta dimensión podría ser el limite de una opción válida.

En conclusión:

**Límite de dimensión fuerza bruta**: 10

**Límite de dimensión ramificación y poda**: 12